<a href="https://colab.research.google.com/github/ParkEunHyeok/Project_Kermit/blob/main/GPT2/Kermit_GPT2_ChatbotTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm
!pip install kobert-transformers==0.4.1
!pip install kogpt2-transformers==0.3.0
!pip install transformers==3.0.2
!pip install torch
!pip install tokenizers==0.8.1rc1
!pip install kss

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 636 kB 46.2 MB/s 
     |████████████████████████████████| 895 kB 43.4 MB/s 
     |████████████████████████████████| 3.3 MB 37.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 769 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 20.1 MB/s 
     |████████████████████████████████| 3.0 MB 43.5 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.10.3
    Uninstalling transformers-4.10.3:
      Successfully uninstalled transformers-4.10.3
     |████████████████████████████████| 4

In [2]:
# 구글 드라이브 연결
import os
import numpy as np
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/HelloNewWorld/gpt"

Mounted at /content/gdrive/


In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
from kogpt2_transformers import get_kogpt2_tokenizer, get_kogpt2_model

In [4]:
class DialogKoGPT2(nn.Module):
  def __init__(self):
    super(DialogKoGPT2, self).__init__()
    self.kogpt2 = get_kogpt2_model()

  def generate(self,
               input_ids,
               do_sample=True,
               max_length= 60,
               top_p=0.92,
               top_k=50,
               temperature= 0.6,
               no_repeat_ngram_size =None,
               num_return_sequences=3,
               early_stopping=False,
               ):
    return self.kogpt2.generate(input_ids,
               do_sample=do_sample,
               max_length=max_length,
               top_p = top_p,
               top_k=top_k,
               temperature=temperature,
               no_repeat_ngram_size= no_repeat_ngram_size,
               num_return_sequences=num_return_sequences,
               early_stopping = early_stopping,
              )

  def forward(self, input, labels = None):
    if labels is not None:
      outputs = self.kogpt2(input, labels=labels)
    else:
      outputs = self.kogpt2(input)

    return outputs

In [6]:
root_path=path
data_path = f"{root_path}/data/wellness_dialog_for_autoregressive.txt"
checkpoint_path =f"{root_path}/checkpoint"
# save_ckpt_path = f"{checkpoint_path}/kogpt2-wellnesee-auto-regressive-20210922-add-chatbotdata.pth"
save_ckpt_path = f"{checkpoint_path}/kogpt2-wellness-auto-regressive.pth"

ctx = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(ctx)

# 저장한 Checkpoint 불러오기
checkpoint = torch.load(save_ckpt_path, map_location=device)

model = DialogKoGPT2()
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

tokenizer = get_kogpt2_tokenizer()

count = 0
output_size = 200 # 출력하고자 하는 토큰 갯수

Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/510M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/920k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/849k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

In [8]:
while 1:
#for i in range(5):
  sent = input('Question: ')  # '요즘 기분이 우울한 느낌이에요'
  tokenized_indexs = tokenizer.encode(sent)

  input_ids = torch.tensor([tokenizer.bos_token_id,]  + tokenized_indexs +[tokenizer.eos_token_id]).unsqueeze(0)
  # set top_k to 50
  sample_output = model.generate(
      input_ids=input_ids
      )


  print("Answer: " + tokenizer.decode(sample_output[0].tolist()[len(tokenized_indexs)+1:],skip_special_tokens=True))
  print(100 * '-')
  # print(sample_output.shape)

# for s in kss.split_sentences(sent):
#     print(s)

Question: 요새 힘들어요


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 잘 버티고 있어요. 조금 힘들더라도 조금만 기운을 내요. 파이팅! 오늘도 파이팅! 오늘도 파이팅! 힘내세요.안녕히 주무세요.안녕히 주무세요.안녕
----------------------------------------------------------------------------------------------------
Question: 선임이 괴롭혀서 삶이 힘들어요


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 이뤄질 수 없는 일. 하지만 오늘을 낭비하지 말아요. 그렇지 않으면 망해요. 할 수 있다면 무엇이든 해보세요. 하고 싶은 게 너무 많다는 걸요. 지금 하는 일
----------------------------------------------------------------------------------------------------
Question: 자살생각이들어요


...

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 스트레스가 많으면 정말 힘들죠. 해소할 수 있는 방법을 찾아 보는 건 어떨까요? 심호흡을 하자고 얘기해주세요. 괜찮으시다면요. 잠깐
----------------------------------------------------------------------------------------------------
Question: 안녕하세요


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Answer: 안녕하세요.안녕하세요.안녕히 주무세요.안녕히 계세요. 그러면 저도 같이 웃어볼까요. 웃음이 많은 사람이 되길 바라요.안녕히 계세요. 저도 같이
----------------------------------------------------------------------------------------------------


KeyboardInterrupt: ignored